## 查询地址普通交易列表

In [56]:
import requests
import pandas as pd
import pytz
from datetime import datetime
import time
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)


def processData(apiKey, chainShortName, address, beginTime=None, endTime=None, timezone=None):
    # Base URL and API endpoint
    base_url = "https://www.oklink.com/api/v5/explorer/address/normal-transaction-list"

    # Initialize a DataFrame to hold all transactions
    all_transactions = pd.DataFrame()

    page = 1
    limit = 100  # limit to 100 transactions at maximum per request
    
    while True:
        # Construct query parameters
        query_params = {
            "chainShortName": chainShortName,
            "address": address,
            "page": page,
            "limit": limit
        }

        # Send the API request
        response = requests.get(base_url, headers={'Ok-Access-Key': apiKey}, params=query_params)

        # Ensure the request was successful
        if response.status_code == 200:
            response_data = response.json()
            # Extract the transactions data
            if "data" in response_data and len(response_data["data"]) > 0:
                transactions = response_data["data"][0].get("transactionList", [])
            else:
                transactions = []
            
            if not transactions:
                break  # Exit the loop if there are no transactions
            
            # Convert the transactions data to a DataFrame
            df = pd.json_normalize(transactions)
            
            # Adding custom columns based on specific data
            if not df.empty:
                df.rename(columns={'txId': 'txHash', 'symbol': 'currency', 'transactionTime': 'datetime'}, inplace=True)
                df['datetime'] = pd.to_datetime(pd.to_numeric(df['datetime'], errors='coerce'), unit='ms').dt.strftime('%Y-%m-%dT%H:%M:%SZ')
                df['contactPlatformSlug'] = ''
                df['direction'] = df['to'].apply(lambda x: 'IN' if x == address else 'OUT')
                df['amount'] = df['amount'].astype(float)
                df['contactIdentity'] = df.apply(lambda row: row['to'] if row['from'] == address else row['from'], axis=1)
                df['type'] = df.apply(lambda row: 'CHAIN_TRANSFER_IN' if row['to'] == address else 'CHAIN_TRANSFER_OUT', axis=1)
                df = df[df['state'] == 'success']
                df = df[['type', 'txHash', 'datetime', 'contactIdentity',
                      'contactPlatformSlug', 'direction', 'currency', 'amount', 'txFee']]
                df['txFee'] = df['txFee'].astype(float)
                all_transactions = pd.concat([all_transactions, df], ignore_index=True)
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return pd.DataFrame()  # return an empty DataFrame in case of an error

        page += 1  # Move to the next page

    # 过滤时间
    if not all_transactions.empty:
        if beginTime and endTime and timezone:
            timezone = pytz.timezone(timezone)
            beginTime_tr = timezone.localize(datetime.strptime(beginTime, '%Y-%m-%d').replace(hour=0, minute=0, second=0)).astimezone(pytz.utc).strftime('%Y-%m-%dT%H:%M:%SZ')
            endTime_tr = timezone.localize(datetime.strptime(endTime, '%Y-%m-%d').replace(hour=23, minute=59, second=59)).astimezone(pytz.utc).strftime('%Y-%m-%dT%H:%M:%SZ')
        
            all_transactions = all_transactions[(all_transactions['datetime'] >= beginTime_tr) & (all_transactions['datetime'] <= endTime_tr)]
        
        # Reset index and drop duplicates just in case
        all_transactions.drop_duplicates(subset=['txHash'], inplace=True)
        all_transactions.reset_index(drop=True, inplace=True)
    
    return all_transactions

In [57]:
# Example usage
apiKey = "d4a4be25-10a1-486a-ac67-7dcd9fa1d7db"
chainShortName = "base"
address = "0x9b6899d37d3200a9eae9f0e24765e8cc2057856d"

df = processData(apiKey, chainShortName, address, beginTime = '2024-01-01', endTime = '2024-07-09', timezone = 'UTC')
df

,type,txHash,datetime,contactIdentity,contactPlatformSlug,direction,currency,amount,txFee
0,CHAIN_TRANSFER_OUT,0xa92cdbea74a8113c6a095883cf005936d82513e9cae5...,2024-07-09T03:54:55Z,0xa70e4d313fba81280137483a1b6d8f738db77a63,,OUT,ETH,0.000000,4.202649e-07
1,CHAIN_TRANSFER_OUT,0x072bf1cb0b11d7a8889b14b6a3bb789e81a193627080...,2024-07-09T03:54:29Z,0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7,,OUT,ETH,0.000075,7.638165e-07
2,CHAIN_TRANSFER_OUT,0xff63797c07a814af2af42a05a53983f216a216db5fa3...,2024-07-09T03:54:11Z,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,,OUT,ETH,0.000010,5.301356e-07
3,CHAIN_TRANSFER_OUT,0x9013ef489346b03f0f33f7145924b91eb1b58afa1361...,2024-07-09T03:48:49Z,0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7,,OUT,ETH,0.000075,7.388703e-07
4,CHAIN_TRANSFER_OUT,0xb7fe7a3d04c942694c11115ebabd24e2c08189ea9550...,2024-07-09T03:44:09Z,0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7,,OUT,ETH,0.000075,7.332585e-07
5,CHAIN_TRANSFER_OUT,0x3a7b87272aca04e830cf2cdddc4de7959644f747bc2c...,2024-07-09T03:36:41Z,0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7,,OUT,ETH,0.000075,7.338420e-07
6,CHAIN_TRANSFER_OUT,0xeda211d3332af0814c9f335a36b65d266e0fffa3d525...,2024-07-09T03:36:13Z,0x8f44fd754285aa6a2b8b9b97739b79746e0475a7,,OUT,ETH,0.000000,6.741303e-07
7,CHAIN_TRANSFER_OUT,0xe20ca97d0d2f1e92a56918e82fbf1e7de312593843c4...,2024-06-19T03:59:15Z,0x8f44fd754285aa6a2b8b9b97739b79746e0475a7,,OUT,ETH,0.000000,1.210767e-06
8,CHAIN_TRANSFER_OUT,0x4363be8655b650114e4e1c5117918ddf041359f11204...,2024-06-19T03:58:49Z,0x4ed4e862860bed51a9570b96d89af5e1b0efefed,,OUT,ETH,0.000000,2.731592e-07
9,CHAIN_TRANSFER_OUT,0xb47b806324016e9994c65b2f079319da5e2e939f2254...,2024-06-05T11:03:51Z,0x5e809a85aa182a9921edd10a4163745bb3e36284,,OUT,ETH,0.004750,1.671570e-07


In [58]:
temp = df
temp.loc[temp['direction'] == 'OUT', 'amount'] *= -1

total_amount = temp['amount'].sum()
# 原始输出有txFee的部分，但是算上了还是对不上
# fee = temp['txFee'].sum()
# total_amount += fee

print("普通交易Total amount:", total_amount)

Total amount: -0.030286700385983915


## 查询地址内部交易列表

In [35]:
import requests
import pandas as pd
import pytz
from datetime import datetime
import time
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
def processDataIn(apiKey, chainShortName, address, beginTime=None, endTime=None, timezone=None):
    # Base URL and API endpoint
    base_url = "https://www.oklink.com/api/v5/explorer/address/internal-transaction-list"

    # Initialize a DataFrame to hold all transactions
    all_transactions = pd.DataFrame()

    page = 1
    limit = 100  # limit to 100 transactions at maximum per request
    
    while True:
        # Construct query parameters
        query_params = {
            "chainShortName": chainShortName,
            "address": address,
            "page": page,
            "limit": limit
        }

        # Send the API request
        response = requests.get(base_url, headers={'Ok-Access-Key': apiKey}, params=query_params)

        # Ensure the request was successful
        if response.status_code == 200:
            response_data = response.json()
            # Extract the transactions data
            if "data" in response_data and len(response_data["data"]) > 0:
                transactions = response_data["data"][0].get("transactionList", [])
            else:
                transactions = []
            
            if not transactions:
                break  # Exit the loop if there are no transactions
            
            # Convert the transactions data to a DataFrame
            df = pd.json_normalize(transactions)
            
            # Adding custom columns based on specific data
            if not df.empty:
                df.rename(columns={'txId': 'txHash', 'symbol': 'currency', 'transactionTime': 'datetime'}, inplace=True)
                df['datetime'] = pd.to_datetime(pd.to_numeric(df['datetime'], errors='coerce'), unit='ms').dt.strftime('%Y-%m-%dT%H:%M:%SZ')
                df['contactPlatformSlug'] = ''
                df['direction'] = df['to'].apply(lambda x: 'IN' if x == address else 'OUT')
                df['amount'] = df['amount'].astype(float)
                df['contactIdentity'] = df.apply(lambda row: row['to'] if row['from'] == address else row['from'], axis=1)
                df['type'] = df.apply(lambda row: 'CHAIN_TRANSFER_IN' if row['to'] == address else 'CHAIN_TRANSFER_OUT', axis=1)
                
                df = df[['type', 'txHash', 'datetime', 'contactIdentity',
                      'contactPlatformSlug', 'direction', 'currency', 'amount']]
                
                all_transactions = pd.concat([all_transactions, df], ignore_index=True)
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return pd.DataFrame()  # return an empty DataFrame in case of an error

        page += 1  # Move to the next page

    # 过滤时间
    if not all_transactions.empty:
        if beginTime and endTime and timezone:
            timezone = pytz.timezone(timezone)
            beginTime_tr = timezone.localize(datetime.strptime(beginTime, '%Y-%m-%d').replace(hour=0, minute=0, second=0)).astimezone(pytz.utc).strftime('%Y-%m-%dT%H:%M:%SZ')
            endTime_tr = timezone.localize(datetime.strptime(endTime, '%Y-%m-%d').replace(hour=23, minute=59, second=59)).astimezone(pytz.utc).strftime('%Y-%m-%dT%H:%M:%SZ')
        
            all_transactions = all_transactions[(all_transactions['datetime'] >= beginTime_tr) & (all_transactions['datetime'] <= endTime_tr)]
        
        # Reset index and drop duplicates just in case
        all_transactions.drop_duplicates(subset=['txHash', 'contactIdentity', 'amount'])
        all_transactions.reset_index(drop=True, inplace=True)
    
    return all_transactions

In [36]:
apiKey = "d4a4be25-10a1-486a-ac67-7dcd9fa1d7db"
chainShortName = "base"
address = "0x9b6899d37d3200a9eae9f0e24765e8cc2057856d"

df_in = processDataIn(apiKey, chainShortName, address, beginTime = '2024-01-01', endTime = '2024-07-10', timezone = 'UTC')
df_in

,type,txHash,datetime,contactIdentity,contactPlatformSlug,direction,currency,amount
0,CHAIN_TRANSFER_IN,0x7543d3d9b22c852f9983cbcccde3ab7fb5d672553b24...,2024-07-09T03:47:55Z,0x2f59e9086ec8130e21bd052065a9e6b2497bb102,,IN,ETH,0.000836
1,CHAIN_TRANSFER_IN,0x498f1c0b8cd9157d97aed4741f05b7ab5ef630e0a78b...,2024-05-31T02:34:37Z,0x09aea4b2242abc8bb4bb78d537a67a245a7bec64,,IN,ETH,0.019996
2,CHAIN_TRANSFER_IN,0xbb8a2e9c272bf3e00211ad038cc84181f8956a2e338c...,2024-04-21T05:15:59Z,0xe8c5b8488feafb5df316be73ede3bdc26571a773,,IN,ETH,0.000998
3,CHAIN_TRANSFER_IN,0x6c0c2ec6da86b93128e22f5cf705c7379b4c079e07ef...,2024-04-21T02:37:25Z,0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae,,IN,ETH,0.000000
4,CHAIN_TRANSFER_IN,0x49ffc0103d06f2065b6ef2303e57197a84a310ab6fd4...,2024-04-19T11:39:49Z,0x09aea4b2242abc8bb4bb78d537a67a245a7bec64,,IN,ETH,0.000090
5,CHAIN_TRANSFER_IN,0xb723bcb9e873f16191ca3caaca07500f6f72cc907fcb...,2024-04-19T10:21:55Z,0x6a1431bb23e08e3209dae3130b441863855fc14b,,IN,ETH,0.000218
6,CHAIN_TRANSFER_IN,0xb723bcb9e873f16191ca3caaca07500f6f72cc907fcb...,2024-04-19T10:21:55Z,0x6a1431bb23e08e3209dae3130b441863855fc14b,,IN,ETH,0.000183
7,CHAIN_TRANSFER_IN,0xb723bcb9e873f16191ca3caaca07500f6f72cc907fcb...,2024-04-19T10:21:55Z,0x6a1431bb23e08e3209dae3130b441863855fc14b,,IN,ETH,0.000106
8,CHAIN_TRANSFER_IN,0x0116f2e5eeeafaaf7a1deb75b61e9966aa47f57d7128...,2024-04-19T08:13:47Z,0x09aea4b2242abc8bb4bb78d537a67a245a7bec64,,IN,ETH,0.003715
9,CHAIN_TRANSFER_IN,0xffec3ecc6ed7448fb2c2fd1cb14abf2531897ecb5efc...,2024-04-18T07:39:31Z,0x224d8fd7ab6ad4c6eb4611ce56ef35dec2277f03,,IN,ETH,0.004997


In [60]:
temp = df_in
temp.loc[temp['direction'] == 'OUT', 'amount'] *= -1

total_amount_in = temp['amount'].sum()

# 打印结果
print("内部交易Total amount:", total_amount_in)

内部交易Total amount: 0.0321367485950947


In [61]:
amount = total_amount + total_amount_in
print("普通+内部Total amount:", amount)

普通+内部Total amount: 0.0018500482091107877


## 查询地址代币交易列表

In [41]:
import requests
import pandas as pd
import pytz
from datetime import datetime
import time
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
def processDataToken(apiKey, chainShortName, address, protocolType, beginTime=None, endTime=None, timezone=None):
    # Base URL and API endpoint
    base_url = "https://www.oklink.com/api/v5/explorer/address/token-transaction-list"

    # Initialize a DataFrame to hold all transactions
    all_transactions = pd.DataFrame()

    page = 1
    limit = 100  # limit to 100 transactions at maximum per request
    
    while True:
        # Construct query parameters
        query_params = {
            "chainShortName": chainShortName,
            "address": address,
            "protocolType": protocolType,
            "page": page,
            "limit": limit
        }

        # Send the API request
        response = requests.get(base_url, headers={'Ok-Access-Key': apiKey}, params=query_params)

        # Ensure the request was successful
        if response.status_code == 200:
            response_data = response.json()
            # Extract the transactions data
            if "data" in response_data and len(response_data["data"]) > 0:
                transactions = response_data["data"][0].get("transactionList", [])
            else:
                transactions = []
            
            if not transactions:
                break  # Exit the loop if there are no transactions
            
            # Convert the transactions data to a DataFrame
            df = pd.json_normalize(transactions)
            
            # Adding custom columns based on specific data
            if not df.empty:
                df.rename(columns={'txId': 'txHash', 'symbol': 'currency', 'transactionTime': 'datetime'}, inplace=True)
                df['datetime'] = pd.to_datetime(pd.to_numeric(df['datetime'], errors='coerce'), unit='ms').dt.strftime('%Y-%m-%dT%H:%M:%SZ')
                df['contactPlatformSlug'] = ''
                df['direction'] = df['to'].apply(lambda x: 'IN' if x == address else 'OUT')
                df['amount'] = df['amount'].astype(float)
                df['contactIdentity'] = df.apply(lambda row: row['to'] if row['from'] == address else row['from'], axis=1)
                df['type'] = df.apply(lambda row: 'CHAIN_TRANSFER_IN' if row['to'] == address else 'CHAIN_TRANSFER_OUT', axis=1)
                
                df = df[['type', 'txHash', 'datetime', 'contactIdentity',
                      'contactPlatformSlug', 'direction', 'currency', 'amount']]
                
                all_transactions = pd.concat([all_transactions, df], ignore_index=True)
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return pd.DataFrame()  # return an empty DataFrame in case of an error

        page += 1  # Move to the next page

    # 过滤时间
    if not all_transactions.empty:
        if beginTime and endTime and timezone:
            timezone = pytz.timezone(timezone)
            beginTime_tr = timezone.localize(datetime.strptime(beginTime, '%Y-%m-%d').replace(hour=0, minute=0, second=0)).astimezone(pytz.utc).strftime('%Y-%m-%dT%H:%M:%SZ')
            endTime_tr = timezone.localize(datetime.strptime(endTime, '%Y-%m-%d').replace(hour=23, minute=59, second=59)).astimezone(pytz.utc).strftime('%Y-%m-%dT%H:%M:%SZ')
        
            all_transactions = all_transactions[(all_transactions['datetime'] >= beginTime_tr) & (all_transactions['datetime'] <= endTime_tr)]
        
        # Reset index and drop duplicates just in case
        all_transactions.drop_duplicates(subset=['txHash', 'contactIdentity', 'amount'])
        all_transactions.reset_index(drop=True, inplace=True)
    
    return all_transactions

apiKey = "d4a4be25-10a1-486a-ac67-7dcd9fa1d7db"
chainShortName = "base"
address = "0x9b6899d37d3200a9eae9f0e24765e8cc2057856d"
protocolType = "token_20"

df_token = processDataToken(apiKey, chainShortName, address, protocolType, beginTime = '2024-01-01', endTime = '2024-07-10', timezone = 'UTC')
df_token

,type,txHash,datetime,contactIdentity,contactPlatformSlug,direction,currency,amount
0,CHAIN_TRANSFER_IN,0xea89f975dd20e8460ffde3302358267b0b66e0f4d336...,2024-07-09T03:56:05Z,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,,IN,UNI visit getunic.pro to swap ETH,1.200000e+03
1,CHAIN_TRANSFER_IN,0xff63797c07a814af2af42a05a53983f216a216db5fa3...,2024-07-09T03:54:11Z,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,,IN,USDC,3.062000e-02
2,CHAIN_TRANSFER_OUT,0xeda211d3332af0814c9f335a36b65d266e0fffa3d525...,2024-07-09T03:36:13Z,0x39c0926455682a2afb5680045f309848c350de34,,OUT,DEGEN,1.000000e-01
3,CHAIN_TRANSFER_IN,0xeda211d3332af0814c9f335a36b65d266e0fffa3d525...,2024-07-09T03:36:13Z,0x0000000000000000000000000000000000000000,,IN,sDEGEN,1.000000e-01
4,CHAIN_TRANSFER_IN,0x90b75dff09b1e5ebed40aba4991239f340dfd7c7a27d...,2024-07-02T17:27:45Z,0xd152f549545093347a162dce210e7293f1452150,,IN,$BEAGL,1.000000e+03
5,CHAIN_TRANSFER_OUT,0xe20ca97d0d2f1e92a56918e82fbf1e7de312593843c4...,2024-06-19T03:59:15Z,0x39c0926455682a2afb5680045f309848c350de34,,OUT,DEGEN,1.000000e-01
6,CHAIN_TRANSFER_IN,0xe20ca97d0d2f1e92a56918e82fbf1e7de312593843c4...,2024-06-19T03:59:15Z,0x0000000000000000000000000000000000000000,,IN,sDEGEN,1.000000e-01
7,CHAIN_TRANSFER_IN,0x787878be5ded8ac1d8e4a27852a58d0bf588bf64b612...,2024-06-09T21:52:25Z,0x4ed4e862860bed51a9570b96d89af5e1b0efefed,,IN,Claim at: airdrop-degen.com,4.500000e+06
8,CHAIN_TRANSFER_IN,0xdd12b832ddc6cbd2ba19dc89600731afee07da16a24d...,2024-06-05T11:03:57Z,0x20fe51a9229eef2cf8ad9e89d91cab9312cf3b7a,,IN,Acces rewards on https://stkether.com,1.000000e+00
9,CHAIN_TRANSFER_IN,0xcc9de3a000cac3da7724138b02f2a0cf0756782581ae...,2024-06-03T01:55:49Z,0x9c4de817d24e54a68d5d6253ad2e93d866732376,,IN,DEGEN,2.000000e+01


这个结果比网站上多了一条时间为'2024-06-09T21:52:25Z'的数据

In [62]:
temp = df_token
temp.loc[temp['direction'] == 'OUT', 'amount'] *= -1

total_amount_token = temp['amount'].sum()

# 打印结果
print("token Total amount:", total_amount_token)

token Total amount: 4502226.454149852
